In [ ]:
## imports and object instantiation
import json, time, itertools, sys, threading, psycopg2, pickle, ast
from concurrent.futures import ThreadPoolExecutor, as_completed
import ipywidgets as widgets
import pandas as pd
from IPython.display import clear_output, display, Markdown, HTML
from ipytree import Tree, Node
from psycopg2.extras import RealDictCursor
from psycopg2 import ProgrammingError
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import networkx as nx

conn = psycopg2.connect("dbname=stackql user=stackql host=localhost port=5466")

In [ ]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

In [ ]:
# shared functions
def display_cards(cards_data):
    cards_html = ''
    
    for title, value in cards_data:
        card_template = f"""
        <div style="
            border: 1px solid #e3e3e3;
            border-radius: 4px;
            padding: 20px;
            display: inline-block;
            margin: 5px;
            text-align: center;
            width: 150px;
            background-color: #f7f7f7;">
            <h4 style="margin: 5px 0;">{title}</h4>
            <span style="font-size: 30px; font-weight: bold; color: red;">{value}</span>
        </div>
        """
        cards_html += card_template
    
    display(HTML(cards_html))

def print_overwrite(message):
    clear_output(wait=True)
    print(message)


In [ ]:
# stackql wrapper functions
def run_stackql_query(query, debug=False):
    if debug:
        print_overwrite(f"Executing: {query}...")    
    try:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(query)
            rows = cur.fetchall()
            return pd.DataFrame(rows)
    except Exception as e:
        if debug:
            print(f"Error executing query: {str(e)}")
        return pd.DataFrame()

def run_stackql_queries(queries, debug=False):
    start_time = time.time()
    all_results = []
    
    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        for query in queries:
            if debug:
                print_overwrite(f"Executing: {query}...")
            try:
                cur.execute(query)
                results = cur.fetchall()
                if results:  # check if the result is not empty
                    all_results.extend(results)
            except Exception as e:
                if debug:
                    print(f"Error executing query: {str(e)}")
                else:                
                    continue  # No results for this query, move on to the next one

    df = pd.DataFrame(all_results)

    number_of_rows = df.shape[0]
    elapsed_time = round(time.time() - start_time)
    
    if debug:
        print(f"Found {number_of_rows} rows in {elapsed_time} seconds")

    return df        

In [ ]:
def save_dataframe(df, df_name):
    df.to_pickle("/tmp/%s.pkl" % (df_name))

def save_object(obj, obj_name):
    with open('/tmp/%s.pkl' % (obj_name), 'wb') as f:
        pickle.dump(obj, f)

def load_object(obj_name):
    with open('/tmp/%s.pkl' % (obj_name), 'rb') as f:
        return pickle.load(f)

def load_dataframe(df_name):
    return pd.read_pickle("/tmp/%s.pkl" % (df_name))

In [ ]:
# project vars
servicenetworking_projects = [
]

compute_projects = [
]

storage_projects = [
]

composer_projects = [
]

bigquery_projects = [
]

logging_projects = [
]

dialogflow_projects = [
]

cloudfunctions_projects = [
]

contactcenteraiplatform_projects = [
]

monitoring_projects = [
]

datastudio_projects = [
]

notebooks_projects = [  
]

aiplatform_projects = [
]

artifactregistry_projects = [
]

containerregistry_projects = [
]